In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [3]:
import os


EPOCHS = 1000
BATCH_SIZE = 8
ARCH = "flor"

IMG_SIZE = (128,32, 1)
DATA_ROOT_PATH = "../data"
IMAGES_PATH = os.path.join(DATA_ROOT_PATH, "images", "data")
IMAGES_PATH

'../data/images/data'

In [4]:
vocab = None

with open(os.path.join(DATA_ROOT_PATH, "hindi_vocab.txt"),encoding="utf-8") as f:
  vocab = f.readlines()

idx_to_vocab = {i:value.strip() for i, value in enumerate(vocab)}
vocab_to_idx = {value:key for key, value in idx_to_vocab.items()}

In [5]:
train_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_train.txt"), encoding="utf-8") as f:
  train_data = f.readlines()

In [6]:
valid_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_val.txt"), encoding="utf-8") as f:
  valid_data = f.readlines()

In [7]:
test_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_test.txt"), encoding="utf-8") as f:
  test_data = f.readlines()

In [8]:
test_data1 = None

with open(os.path.join(DATA_ROOT_PATH, "new_te.txt"), encoding="utf-8") as f:
  test_data1 = f.readlines()

In [9]:
charl = None

with open( "charList.txt", encoding="utf-8") as f:
  charl = f.readlines()

In [10]:
charl=charl[0]

In [11]:
charl[9]

'उ'

In [12]:
len(test_data1)

12865

In [13]:
len(train_data)

69803

In [14]:
len(valid_data)

12713

In [15]:
for i in range(4000):
    train_data.append(valid_data[i])

# train data labels

# train data generation

In [16]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp

In [17]:

##label generator
def label_g(l):
    label=idx_to_vocab[l]
    z=[]
    for j in range(27):
        z.append(0)
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z[k]=r
    return z   

In [18]:

##label generator
def label_g1(l):
    label=idx_to_vocab[l]
    z=[]
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z.append(r)
    return z   

In [19]:
train_data[0]

'HindiSeg/train/10/259/1.jpg 10800\n'

In [20]:
import cv2
import numpy as np

In [21]:
s=np.zeros((8, 27))
s

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [22]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp
import cv2

class DataGen(tf.keras.utils.Sequence):
    def __init__(self, data, img_size, batch_size, mode="TRAIN"):
        self.data = data
        self.img_size = img_size
        self.batch_size = batch_size
        self.mode = mode
        
    def __getitem__(self, i):
        start = i * self.batch_size
        end = (i+1) * self.batch_size
        batch_images = np.zeros((self.batch_size, self.img_size[0], self.img_size[1], 1))
        batch_labels = np.zeros((self.batch_size, 27))
        #batch_labels =[[],[],[],[],[],[],[],[]]
        
        for ii, df_index in enumerate(range(start, end)):
            curr_data = self.data[df_index].split()
            curr_img_path = curr_data[0]
            #curr_label = idx_to_vocab[int(curr_data[1])]
            curr_label = label_g(int(curr_data[1]))

            curr_img_path = "/".join(curr_img_path.split("/")[2:])
            curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)

            
            curr_img = pp.preprocess(curr_img_path, self.img_size)
            curr_img=cv2.adaptiveThreshold(curr_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 199, 5)
            #curr_img = tf.image.resize(curr_img, (self.img_size[0], self.img_size[1]), method="nearest")
            #curr_img = curr_img.numpy().reshape((self.img_size[0], self.img_size[1],1))
            batch_images[ii, :, :,0] = curr_img 
            batch_labels[ii,:] = curr_label
                    
        if self.mode == "TRAIN":
          return batch_images, batch_labels
        else:
          return batch_images
        
    def __len__(self):
      return len(self.data) // self.batch_size

In [23]:
train_datagen = DataGen(train_data, IMG_SIZE, 8)

In [24]:
valid_datagen = DataGen(valid_data, IMG_SIZE, 8)

In [25]:
test_datagen = DataGen(test_data1, IMG_SIZE, 10000)

In [26]:
train_datagen[0][1]==train_datagen[1][1]

array([[False, False, False, False, False, False, False, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False,  True,  True,  True,  True,  True,  True,  True,  True,
  

# model


In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.constraints import MaxNorm

from network.layers import FullGatedConv2D, GatedConv2D, OctConv2D
from tensorflow.keras.layers import Conv2D, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.layers import Input, Add, Activation, Lambda, MaxPooling2D, Reshape

In [31]:
from datetime import datetime

In [34]:
output_path = os.path.join("..", "output",str(datetime.now()), ARCH)
target_path = os.path.join(output_path, "checkpoint_weights.hdf5")

In [35]:
datetime.now()

datetime.datetime(2023, 5, 9, 15, 26, 11, 377920)

In [36]:
from network.model import HTRModel

# create and compile HTRModel
model = HTRModel(architecture=ARCH,
                 input_size=IMG_SIZE,
                 vocab_size=110,
                 beam_width=10,
                 stop_tolerance=20,
                 reduce_tolerance=15,
                 reduce_factor=0.1)

model.compile(learning_rate=0.001)
model.summary(output_path, "summary.txt")

callbacks = model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 128, 32, 1)]      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 64, 16, 16)        160       
                                                                 
 p_re_lu_6 (PReLU)           (None, 64, 16, 16)        16        
                                                                 
 batch_normalization_6 (Batc  (None, 64, 16, 16)       112       
 hNormalization)                                                 
                                                                 
 full_gated_conv2d_5 (FullGa  (None, 64, 16, 16)       4640      
 tedConv2D)                                                      
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 16, 32)        4640

In [38]:
start_time = datetime.now()

model.fit(x=train_datagen,
              epochs=EPOCHS,
          validation_data=valid_datagen,
              callbacks=callbacks,
              verbose=1)
total_time = datetime.now() - start_time

print("total time",total_time)

Epoch 1/1000
  49/9225 [..............................] - ETA: 12:32 - loss: 26.8182

KeyboardInterrupt: 

In [31]:
xt=test_datagen[0][0]
yt=test_datagen[0][1]

In [32]:
predicts, _ = model.predict(x=xt,
                            ctc_decode=True,
                            verbose=1)

Model Predict


2023-05-05 09:28:48.698778: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-05 09:28:48.698813: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.51GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-05 09:28:48.717611: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.91GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-05 09:28:48.717628: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU

1/1 [==============================] - 5s 5s/step
CTC Decode
1/1 [==============================] - 86s 86s/step


/home/shashank/anaconda3/envs/htr2/lib/python3.8/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [33]:
len(predicts[0])

1

In [34]:
predicts[1][0]

[56, 44, 40, 68]

In [35]:
def convert(predicts):
    a=[]
    b=[]
    for i in range(len(predicts)):
        b=[]
        for j in range(len(predicts[i][0])):
            b.append(charl[predicts[i][0][j]])
        a.append(b)
    return a

In [36]:
def convert1(predicts):
    a=[]
    b=[]
    for i in range(len(predicts)):
        b=[]
        for j in range(len(predicts[i])):
            b.append(charl[int(predicts[i][j])])
        a.append(b)
    return a

In [37]:
def trim(yt):
    yt1=[]
    for i in range(len(yt)):
        k=0
        for j in range(26,0,-1):
            if yt[i][j]!=0:
                yt1.append(yt[i][0:j+1])
                k=1
                break
        if k==0:
            yt1.append([0])
    return yt1

In [38]:
yt1=trim(yt)

In [39]:
len(yt1[0])

10

In [40]:
len(yt[0])

27

In [41]:
yt1[0][1]

69.0

In [42]:
predicts1=convert(predicts)

In [43]:
gt=convert1(yt1)

In [44]:
from data import evaluation

In [45]:

import string
import unicodedata
import editdistance
import numpy as np


def ocr_metrics(predicts, ground_truth, norm_accentuation=False, norm_punctuation=False):
    """Calculate Character Error Rate (CER), Word Error Rate (WER) and Sequence Error Rate (SER)"""

    if len(predicts) == 0 or len(ground_truth) == 0:
        return (1, 1, 1)

    cer, wer, ser = [], [], []

    for (pd, gt) in zip(predicts, ground_truth):
        '''pd, gt = pd.lower(), gt.lower()

        if norm_accentuation:
            pd = unicodedata.normalize("NFKD", pd).encode("ASCII", "ignore").decode("ASCII")
            gt = unicodedata.normalize("NFKD", gt).encode("ASCII", "ignore").decode("ASCII")

        if norm_punctuation:
            pd = pd.translate(str.maketrans("", "", string.punctuation))
            gt = gt.translate(str.maketrans("", "", string.punctuation))
	'''
        pd_cer, gt_cer = list(pd), list(gt)
        dist = editdistance.eval(pd_cer, gt_cer)
        cer.append(dist / (max(len(pd_cer), len(gt_cer))))
        '''
        pd_wer, gt_wer = pd, gt
        dist = editdistance.eval(pd_wer, gt_wer)
        wer.append(dist / (max(len(pd_wer), len(gt_wer))))
        
        pd_ser, gt_ser = [pd], [gt]
        dist = editdistance.eval(pd_ser, gt_ser)
        ser.append(dist / (max(len(pd_ser), len(gt_ser))))
        '''
    metrics = [cer]
    metrics = np.mean(metrics, axis=1)

    return metrics

In [46]:
file5=open('florcer1000.txt','w')

In [47]:
evaluate = ocr_metrics(predicts=predicts1,
                                  ground_truth=gt,)
 
print("Calculate Character Error Rate {} ".format(evaluate[0],))
file5.write(str(evaluate[0]))
file5.close()

Calculate Character Error Rate 0.1880140848857025 


In [48]:
predicts1[0]

['ग', 'ो', 'र', '-', 'च', 'ु', 'ा', 'त', 'ी']

In [49]:
predict_path = os.path.join("..", "predict")

In [ ]:
for i in 

In [50]:
ground_t_p=os.path.join("..", "predict",ARCH)

In [51]:
ground_t_p

'../predict/flor'

In [52]:
file1=open('flor1predict_ground','w')

In [53]:
for i in range(len(predicts1)):
    for j in range(len(gt[i])):
        file1.write(gt[i][j])

In [54]:
for i in range(len(predicts1)):
    for j in range(len(gt[i])):
        file1.write(gt[i][j])

In [55]:
for i in range(len(predicts1)):
    for j in range(len(gt[i])):
        file1.write(gt[i][j])
    file1.write('\n')
    for j in range(len(predicts1[i])):
        file1.write(predicts1[i][j])
    file1.write('\n')

In [56]:
file1.close()

In [57]:
file2=open('flor1ground_t','w')

In [58]:
for i in range(len(gt)):
    for j in range(len(gt[i])):
        file2.write(gt[i][j])
    file2.write('\n')

In [59]:
file2.close()

In [60]:
file3=open('flor1predict1','w')

In [61]:
for i in range(len(predicts1)):
    for j in range(len(predicts1[i])):
        file3.write(predicts1[i][j])
    file3.write('\n')

In [62]:
file3.close()

In [63]:
import string
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import string
import numpy as np



In [64]:

def alg4(word1,word2):
  M=[[float('inf')]*(len(word2)+1) for i in range(len(word1)+1)]
          
  #filling last row
  for i in range(len(word2)+1):
    M[len(word1)][i]=len(word2)-i
          
  #filling last column
  for j in range(len(word1)+1):
    M[j][len(word2)]=len(word1)-j
              
  #filling bottom to up manner
          
  for i in range(len(word1)-1,-1,-1):
    for j in range(len(word2)-1,-1,-1):
      if word1[i]==word2[j]:
        M[i][j]=M[i+1][j+1]
      else:
        M[i][j]=1+min(M[i+1][j],M[i][j+1],M[i+1][j+1])

  x,y=0,0
  #print(x,y)
  count=0
  while x<len(M)-1 and y<len(M[0])-1:
    current=M[x][y]
    dia=M[x+1][y+1]
    right=M[x][y+1]
    bottom=M[x+1][y]
    if dia<=right and dia<=bottom and dia<=current:
      if dia==current-1:
        print("Substitution-->",word1[x],"replaced by",word2[y])
        #array[string.printable[:95].find(word1[x]),string.printable[:95].find(word2[y])]=array[string.printable[:95].find(word1[x]),string.printable[:95].find(word2[y])]+1
        sub[dict1[word1[x]]]=sub[dict1[word1[x]]]+1
        subs[dict1[word2[x]]][dict1[word1[x]]]=subs[dict1[word2[x]]][dict1[word1[x]]]+1
        count=count+1
        x=x+1
        y=y+1
      else:
        print("No operation-->",word1[x])
        x=x+1
        y=y+1
      
    elif right<=bottom and right<=current:
      print("Insertion",word2[y])
      ins[dict1[word2[y]]]=ins[dict1[word2[y]]]+1

      count=count+1
      y=y+1
    else:
      print("Deletion",word1[x])
      delete[dict1[word1[x]]]=delete[dict1[word1[x]]]+1
      x=x+1
      count=count+1
  print("total operations",count)
  

In [65]:
delete=[]
ins=[]
sub=[]

for i in range(len(charl)+1):
    delete.append(0)
    ins.append(0)
    sub.append(0)

In [66]:
for i in range(len(gt)):
    s1=""
    for j in range(len(gt[i])):
        s1=s1+gt[i][j]
    s2=""
    for j in range(len(predicts1[i])):
        s2=s2+predicts1[i][j]
    alg4(s2,s1)

No operation--> ग
Substitution--> ो replaced by ै


NameError: name 'dict1' is not defined

In [67]:
file1=open('flor1delete.txt','w')
file2=open('flor1ins.txt','w')
file3=open('flor1sub.txt','w')

In [ ]:
for i in range(len(charl)):
    print(charl[i])
    file1.write(charl[i])
    file1.write('\n')
    file1.write(str(delete[i]))
    file1.write('\n')
    file2.write(charl[i])
    file2.write('\n')
    file2.write(str(ins[i]))
    file2.write('\n')
    file3.write(charl[i])
    file3.write('\n')
    file3.write(str(sub[i]))
    file3.write('\n')
    

In [ ]:
file1.close()
file2.close()
file3.close()

In [ ]:
subs=[]
for i in range(len(charl)):
    z=[]
    for j in range(len(charl)):
        z.append(0)
    subs.append(z)

In [ ]:
for i in range(len(charl)):
    
    for j in range(len(charl)):
        if subs[i][j]!=0:
            print(subs[i][j])

In [ ]:
len(charl)

In [ ]:
dict1={}
for i in range(len(charl)):
    dict1[charl[i]]=i

In [ ]:
dict1

In [ ]:
s1=""
for j in range(len(gt[70])):
    s1=s1+gt[70][j]

In [ ]:
s2=""
for j in range(len(predicts1[70])):
    s2=s2+predicts1[70][j]

In [ ]:
alg4(s1,s2)

In [ ]:
file4=open('flor1subsc.txt','w')

In [ ]:
for i in range(len(subs)):
    for j in range(len(subs)):
        file4.write(str(subs[i][j]))
        file4.write(" ")
    file4.write('\n')

In [ ]:
file4.close()

In [ ]:
gt[70]

In [ ]:
predicts1[70]

# charlist generation
